Google-Colab | T4-GPU

In [1]:
!pip install accelerate peft bitsandbytes transformers trl

In [ ]:
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
import torch

model_name = "NousResearch/Hermes-3-Llama-3.2-3B"
finetune_model = "Hermes-3-Llama-3.2-3B-Finetuned"

# Output folder
output_dir = "./results"

# No of epochs
num_train_epochs =1

# No change params
use_4bit, bnb_4bit_compute_dtype, bnb_4bit_quant_type, use_nested_quant = True, "float16", "nf4", False # To quantization
lora_r, lora_alpha, lora_dropout = 64, 16, 0.1
fp16, bf16 =  False, False
per_device_train_batch_size, per_device_eval_batch_size = 4, 4
gradient_accumulation_steps, gradient_checkpointing, max_grad_norm = 1, True, 0.3
learning_rate, weight_decay, optim = 2e-4, 0.001, "paged_adamw_32bit"
lr_scheduler_type, max_steps, warmup_ratio = "cosine", -1, 0.03
group_by_length, save_steps, logging_steps = True, 0, 25
max_seq_length, packing, device_map = None, False, {"": 0}


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

In [ ]:
from datasets import load_dataset
import numpy as np

dataset_name = "hungsvdut2k2/vietnamese-chat-doctor"

dataset = load_dataset(dataset_name)

def transform_conversation(data):
    reformatted_segments = []
    for x in data['conversation']:
        reformatted_segments.append(f"<|im_start|>{x['role']}\n{x['content']}<|im_end|>\n")

    return {'text': ''.join(reformatted_segments)}

transformed_dataset = dataset['train'].map(transform_conversation)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import Dataset
# train thử trên bộ dataset nhỏ vì giới hạn dung lượng RAM
transformed_dataset_splitted = Dataset.from_list([{'text': text} for text in transformed_dataset['text'][:500]])
len(transformed_dataset_splitted)

500

In [ ]:
# transformed_dataset is there
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,
    HfArgumentParser, TrainingArguments, pipeline, logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model, # model fintune
    train_dataset=transformed_dataset_splitted, # Dataset
    peft_config=peft_config,
    # tokenizer=tokenizer,
    processing_class = tokenizer,
    args=training_arguments
  )

trainer.train()

In [ ]:
# Save trained model
trainer.model.save_pretrained(finetune_model)

In [ ]:
del trainer

# free VRAM
import gc

In [ ]:
# Test model
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "triệu chứng buồn nôn của tôi là bị làm sao vậy bác sĩ"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n")
print(result[0]['generated_text'])

del pipe, model
gc.collect()

<|im_start|> user triệu chứng buồn nôn của tôi là bị làm sao vậy bác sĩ <|im_end|> assistant Chào mừng bạn đến với Chat Doctor. Tôi hiểu mối quan tâm của bạn. Trước khi tôi trả lời câu hỏi của bạn, tôi muốn nói rằng buồn nôn là một triệu chứng rất phổ biến và có thể gây ra bởi nhiều nguyên nhân khác nhau. Nếu bạn bị buồn nôn thường xuyên, bạn nên đến gặp bác sĩ để được kiểm tra. Vì vậy, tôi khuyên bạn nên đến gặp bác sĩ và được kiểm tra kỹ lưỡng. Nếu cần, bạn có thể cần phải được chụp X-quang hoặc được kiểm tra bằng siêu âm. Nếu cần, bạn có thể cần phải được điều trị bằng thuốc. Hy vọng tôi đã trả lời câu hỏi của bạn. Nếu bạn có thêm bất kỳ câu hỏi nào, hãy hỏi. 24 giờ đồng hồ. 2. 3. 4. 5. 6. 7


In [ ]:
# Reload and merge
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, finetune_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [12]:
model.save_pretrained("models/finetune_model/")
tokenizer.save_pretrained("models/tokenizer/")

('models/tokenizer/tokenizer_config.json',
 'models/tokenizer/special_tokens_map.json',
 'models/tokenizer/tokenizer.json')

In [ ]:
# Run text generation pipeline with our next model
prompt = "triệu chứng buồn nôn của tôi là bị làm sao vậy bác sĩ"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n")
print(result[0]['generated_text'])

del pipe, model
gc.collect()

<|im_start|> user triệu chứng buồn nôn của tôi là bị làm sao vậy bác sĩ <|im_end|> assistant Chào mừng bạn đến với Chat Doctor. Tôi hiểu mối quan tâm của bạn. Tôi sẽ giải quyết vấn đề của bạn. Bác sĩ sẽ cho bạn thuốc giảm đau và thuốc chống nôn. Bác sĩ sẽ không cho bạn thuốc giảm đau. Bác sĩ sẽ cho bạn thuốc chống nôn. Bạn có thể uống thuốc chống nôn của mình, sau đó bạn sẽ có thể bắt đầu ăn một số thức ăn nhẹ như cơm và bánh mì. Bạn có thể bắt đầu ăn thức ăn nhẹ như rau, cà rốt, rau diếp, rau chân vịt và cà chua. Bạn cũng có thể bắt đầu ăn các loại thức ăn nhẹ khác như thịt, gà, cá và trứng. Bạn cũng có thể bắt đầu ăn bánh mì, bánh mì và bánh quy. Bạn cũng có thể bắt đầu ăn các loại thức ăn nhẹ khác như rau
